In [1]:
import os
import tarfile
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

# --- 설정 ---
# 업로드할 폴더 경로
SOURCE_FOLDER = 'models/Qwen3_8B-fp8-filtered_full-tune'
# 인증 정보 파일 (가지고 계신 credentials.json 파일)
CLIENT_SECRETS_FILE = 'client_secrets.json' 
# --- 설정 종료 ---

def compress_folder(folder_path):
    """지정된 폴더를 .tar.gz 파일로 압축합니다."""
    if not os.path.isdir(folder_path):
        print(f"오류: '{folder_path}' 폴더를 찾을 수 없습니다.")
        return None
        
    archive_name = f"{os.path.basename(folder_path)}.tar.gz"
    print(f"'{folder_path}' 폴더를 '{archive_name}' 파일로 압축하는 중...")
    
    with tarfile.open(archive_name, 'w:gz') as tar:
        tar.add(folder_path, arcname=os.path.basename(folder_path))
        
    print("압축 완료.")
    return archive_name

def get_drive_service():
    """PyDrive2를 사용하여 Google Drive 서비스 객체를 인증하고 반환합니다."""
    gauth = GoogleAuth()
    
    # settings.yaml 파일이 아닌, 직접 client_secrets.json 경로를 지정합니다.
    gauth.settings['client_config_file'] = CLIENT_SECRETS_FILE
    
    # 로컬 웹서버를 통한 인증
    gauth.LocalWebserverAuth()
    
    return GoogleDrive(gauth)

def upload_folder_recursively(local_folder_path, parent_folder_id, drive_service):
    """로컬 폴더를 재귀적으로 순회하며 구글 드라이브에 업로드합니다."""
    
    # 현재 로컬 폴더 이름으로 구글 드라이브에 폴더 생성
    folder_metadata = {
        'title': os.path.basename(local_folder_path),
        'parents': [{'id': parent_folder_id}],
        'mimeType': 'application/vnd.google-apps.folder'
    }
    drive_folder = drive_service.CreateFile(folder_metadata)
    drive_folder.Upload()
    print(f"폴더 생성: '{drive_folder['title']}' (ID: {drive_folder['id']})")

    # 로컬 폴더 내의 파일과 하위 폴더 순회
    for item_name in os.listdir(local_folder_path):
        local_item_path = os.path.join(local_folder_path, item_name)

        if os.path.isdir(local_item_path):
            # 하위 폴더인 경우 재귀 호출
            upload_folder_recursively(local_item_path, drive_folder['id'], drive_service)
        else:
            # 파일인 경우 업로드
            file_metadata = {
                'title': item_name,
                'parents': [{'id': drive_folder['id']}]
            }
            gfile = drive_service.CreateFile(file_metadata)
            gfile.SetContentFile(local_item_path)
            print(f"  파일 업로드 중: '{item_name}' -> '{drive_folder['title']}'")
            gfile.Upload()

def main():
    """메인 실행 함수"""
    if not os.path.isdir(SOURCE_FOLDER):
        print(f"오류: 원본 폴더 '{SOURCE_FOLDER}'를 찾을 수 없습니다.")
        return

    try:
        drive_service = get_drive_service()
        print(f"'{SOURCE_FOLDER}' 폴더를 구글 드라이브에 업로드 시작합니다...")
        
        # 'root'는 내 드라이브의 최상위 위치를 의미합니다.
        upload_folder_recursively(SOURCE_FOLDER, 'root', drive_service)
        
        print("\n모든 업로드가 완료되었습니다.")

    except Exception as e:
        print(f"작업 중 오류 발생: {e}")

main() 

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=737461501870-4jph467fongiahds638dp74lo1ac99k5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=online&response_type=code

Authentication successful.
'models/Qwen3_8B-fp8-filtered_full-tune' 폴더를 구글 드라이브에 업로드 시작합니다...
폴더 생성: 'Qwen3_8B-fp8-filtered_full-tune' (ID: 1bBzD4JpUiZg0gg884b7wZuavWcGGtDKu)
  파일 업로드 중: 'config.json' -> 'Qwen3_8B-fp8-filtered_full-tune'
  파일 업로드 중: 'generation_config.json' -> 'Qwen3_8B-fp8-filtered_full-tune'
  파일 업로드 중: 'model-00001-of-00004.safetensors' -> 'Qwen3_8B-fp8-filtered_full-tune'
  파일 업로드 중: 'model-00002-of-00004.safetensors' -> 'Qwen3_8B-fp8-filtered_full-tune'
  파일 업로드 중: 'model-00003-of-00004.safetensors' -> 'Qwen3_8B-fp8-filtered_full-tune'
  파일 업로드 중: 'model-00004-of-00004.safetensors' -> 'Qwen3_8B-fp8-filtered_full-tune'
  파일 업로드 중: 'model.safetensors.index.json' -